In [8]:
# Working with Pandas DataFrames

import json
import pandas as pd

def dumps(obj: dict) -> str:
    """Serialize dict-of-list-of-dicts to TOON."""
    if not isinstance(obj, dict) or len(obj) != 1:
        raise ValueError("Top-level object must contain exactly one key")

    name = list(obj.keys())[0]
    rows = obj[name]

    fields = list(rows[0].keys()) if rows else []
    header = ",".join(fields)
    count = len(rows)

    lines = []
    for r in rows:
        vals = []
        for f in fields:
            v = r[f]
            if isinstance(v, bool):
                v = "true" if v else "false"
            vals.append(str(v))
        lines.append("  " + ",".join(vals))

    return f"{name}[{count}]{{{header}}}:\n" + ("\n".join(lines) if lines else "")

def parse(text: str) -> dict:
    """Parse TOON array format back into Python dict."""
    lines = [l.strip() for l in text.strip().splitlines() if l.strip()]
    if not lines:
        return {}

    header = lines[0]

    name = header[:header.find("[")].strip()
    field_block = header[header.find("{")+1 : header.find("}")]
    fields = [f.strip() for f in field_block.split(",")]

    items = []
    for row in lines[1:]:
        parts = row.split(",")
        rec = {}
        for f, v in zip(fields, parts):
            v = v.strip()
            if v.replace(".", "", 1).isdigit():
                v = float(v) if "." in v else int(v)
            elif v in ("true", "false"):
                v = (v == "true")
            rec[f] = v
        items.append(rec)

    return {name: items}

def json_to_toon(json_text: str) -> str:
    """Convert JSON string to TOON."""
    return dumps(json.loads(json_text))

def toon_to_json(toon_text: str) -> str:
    """Convert TOON back to JSON string."""
    return json.dumps(parse(toon_text), indent=2)

def dataframe_to_toon(df: pd.DataFrame, name="table") -> str:
    """Convert a Pandas DataFrame into TOON format."""
    records = df.to_dict(orient="records")
    return dumps({name: records})


import pandas as pd
#from toon_format import dataframe_to_toon

df = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['Site A', 'Site B', 'Site C'],
    'score': [95.5, 87.3, 92.1],
    'passed': [True, True, True]
})

toon_data = dataframe_to_toon(df, name='students')
print(toon_data)


students[3]{id,name,score,passed}:
  1,Site A,95.5,true
  2,Site B,87.3,true
  3,Site C,92.1,true
